##Question 3

In [1]:
import numpy as np

"""
Calculate the forward kinematics of a robotic manipulator.

Args:
- dh_params: List of DH parameters for each link in the format [theta, d, a, alpha].
- joint_types: List of joint types ('R' for revolute, 'P' for prismatic).

Returns:
- T: The transformation matrix representing the end-effector position.
"""

def forward_kinematics(dh_params, joint_types):
    T = np.identity(4)  # Initialize transformation matrix

    for i in range(len(dh_params)):
        theta, d, a, alpha = dh_params[i]
        if joint_types[i] == 'R':  # Revolute joint
            A_i = np.array([[np.cos(theta), -np.sin(theta) * np.cos(alpha), np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
                            [np.sin(theta), np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
                            [0, np.sin(alpha), np.cos(alpha), d],
                            [0, 0, 0, 1]])
        elif joint_types[i] == 'P':  # Prismatic joint
            A_i = np.array([[1, 0, 0, a],
                            [0, 1, 0, d],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]])
        else:
            raise ValueError("Invalid joint type. Use 'R' for revolute or 'P' for prismatic.")

        T = T.dot(A_i)

    return T

"""
Calculate the manipulator Jacobian for a robotic manipulator.

Args:
- dh_params: List of DH parameters for each link in the format [theta, d, a, alpha].
- joint_types: List of joint types ('R' for revolute, 'P' for prismatic).

Returns:
- J: The manipulator Jacobian matrix.
"""

def manipulator_jacobian(dh_params, joint_types):
    n = len(dh_params)  # Number of joints
    J = np.zeros((6, n))  # Initialize Jacobian matrix

    T = forward_kinematics(dh_params, joint_types)

    for i in range(n):
        Z_i = T[:3, 2]
        P_i = T[:3, 3]
        if joint_types[i] == 'R':  # Revolute joint
            J[:, i] = np.concatenate((np.cross(Z_i, (P_i - T[:3, 3])), Z_i))
        elif joint_types[i] == 'P':  # Prismatic joint
            J[:, i] = np.concatenate((Z_i, np.zeros(3)))

        # Update transformation matrix for the next joint
        theta, d, a, alpha = dh_params[i]
        if joint_types[i] == 'R':
            delta_theta = theta
        elif joint_types[i] == 'P':
            delta_theta = 0
        A_i = np.array([[np.cos(delta_theta), -np.sin(delta_theta) * np.cos(alpha), np.sin(delta_theta) * np.sin(alpha), a * np.cos(delta_theta)],
                        [np.sin(delta_theta), np.cos(delta_theta) * np.cos(alpha), -np.cos(delta_theta) * np.sin(alpha), a * np.sin(delta_theta)],
                        [0, np.sin(alpha), np.cos(alpha), d],
                        [0, 0, 0, 1]])
        T = T.dot(A_i)

    return J


"""
Calculate the end-effector velocity for a robotic manipulator.

Args:
- dh_params: List of DH parameters for each link in the format [theta, d, a, alpha].
- joint_types: List of joint types ('R' for revolute, 'P' for prismatic).
- joint_velocities: List of joint velocities (angular or linear, depending on joint type).

Returns:
- ee_velocity: The end-effector velocity vector.
"""
""
def end_effector_velocity(dh_params, joint_types, joint_velocities):

    J = manipulator_jacobian(dh_params, joint_types)
    ee_velocity = J.dot(joint_velocities)

    return ee_velocity

## Question 4


In [2]:
# Example usage for Stanford RRP configuration
stanford_dh_params = [
    [0, 0, 0.2, np.pi/2],
    [0, 0, 0.15, 0],
    [0, 0, 0, np.pi/2]
]

stanford_joint_types = ['R', 'R', 'P']

joint_angles = [np.pi/4, np.pi/4, 0.5]  # Adjust as needed
joint_velocities = [1.0, 1.0, 0.1]  # Adjust as needed

# Calculate end-effector position
T_stanford = forward_kinematics(stanford_dh_params, stanford_joint_types)
ee_position_stanford = T_stanford[:3, 3]

# Calculate manipulator Jacobian
J_stanford = manipulator_jacobian(stanford_dh_params, stanford_joint_types)

# Calculate end-effector velocity
ee_velocity_stanford = end_effector_velocity(stanford_dh_params, stanford_joint_types, joint_velocities)

# Print results
print("Stanford RRP Configuration Results:")
print(f"End-Effector Position: {ee_position_stanford}")
print(f"Manipulator Jacobian:\n{J_stanford}")
print(f"End-Effector Velocity: {ee_velocity_stanford}")

Stanford RRP Configuration Results:
End-Effector Position: [0.35 0.   0.  ]
Manipulator Jacobian:
[[-0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00 -0.0000000e+00 -1.2246468e-16]
 [ 0.0000000e+00  0.0000000e+00 -1.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-1.0000000e+00 -1.2246468e-16  0.0000000e+00]
 [ 6.1232340e-17 -1.0000000e+00  0.0000000e+00]]
End-Effector Velocity: [ 0.0000000e+00 -1.2246468e-17 -1.0000000e-01  0.0000000e+00
 -1.0000000e+00 -1.0000000e+00]


In [3]:
# Example usage for SCARA configuration
scara_dh_params = [
    [0, 0, 0.2, 0],
    [0, 0, 0.15, 0],
    [0, 0, 0, 0]
]

scara_joint_types = ['R', 'R', 'R']

joint_angles = [np.pi/4, np.pi/4, np.pi/4]  # Adjust as needed
joint_velocities = [1.0, 1.0, 1.0]  # Adjust as needed

# Calculate end-effector position
T_scara = forward_kinematics(scara_dh_params, scara_joint_types)
ee_position_scara = T_scara[:3, 3]

# Calculate manipulator Jacobian
J_scara = manipulator_jacobian(scara_dh_params, scara_joint_types)

# Calculate end-effector velocity
ee_velocity_scara = end_effector_velocity(scara_dh_params, scara_joint_types, joint_velocities)

# Print results
print("\nSCARA Configuration Results:")
print(f"End-Effector Position: {ee_position_scara}")
print(f"Manipulator Jacobian:\n{J_scara}")
print(f"End-Effector Velocity: {ee_velocity_scara}")


SCARA Configuration Results:
End-Effector Position: [0.35 0.   0.  ]
Manipulator Jacobian:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 1. 1.]]
End-Effector Velocity: [0. 0. 0. 0. 0. 3.]
